Includes

In [104]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
# import statsmodels.api as sm
# import seaborn as sns
# from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


import cv2
from random import randint
# from PIL import Image
from os import listdir

Compress images

In [105]:
# def compress_image(path : str):
# 	im = Image.open(path)
# 	im.thumbnail((500, 500), Image.LANCZOS)
# 	im.save(path.replace('dataset', 'reduced_dataset'))

# def compress_dataset(path : str):
# 	for num in range(0,6):
# 		men_dir = path + f'men/{num}/'
# 		women_dir = path + f'Women/{num}/'
# 		for file in listdir(men_dir):
# 			compress_image(men_dir + file)
# 		for file in listdir(women_dir):
# 			compress_image(women_dir + file)

# compress_dataset('../../dataset/')

Read training dataset

In [106]:
def read_image(path: str) -> np.array:
	img = cv2.imread(path)
	img = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)
	img_filtered = cv2.inRange(img,(0,136,77),(255,163,140))
	# print(img_filtered_cluttered[5][0])
	# img = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
	# img_masked_shadow = np.bitwise_not(cv2.inRange(img,(0,0,80),(255,55,179)))
	# img_masked_hand = cv2.inRange(img,(5,35,30),(25,255,255))
	# img_filtered = np.bitwise_and(img_masked_shadow,img_masked_hand)
	bucket_size = 10
	img_filtered_cluttered = img_filtered[1:,:].reshape((int(img_filtered[1:,:].shape[0]/bucket_size),-1,bucket_size))
	area = np.array([np.sum(img_filtered.reshape((-1))) / 255])
	histogramH = np.array([np.mean(np.diff(np.sum(img_filtered_cluttered,axis=(1,2))))])
	# print(histogramH.dtype)
	bucket_size = 20
	img_filtered_cluttered = img_filtered.reshape((-1,int(img_filtered.shape[1]/bucket_size),bucket_size))
	histogramV = np.array([np.mean(np.diff(np.sum(img_filtered_cluttered,axis=(0,2))))])
	return np.concatenate((area, histogramH, histogramV, img_filtered.reshape(-1)))

def read_images(directory : str) -> np.array:
	array = np.array([], dtype=np.int8).reshape(0, 140504)
	for num in range(0,6):
		men_dir = directory + f'men/{num}/'
		women_dir = directory + f'Women/{num}/'
		new_men = np.array([np.concatenate((np.asarray([num]), read_image(men_dir + file)), axis=0) for file in listdir(men_dir)])
		new_women = np.array([np.concatenate((np.asarray([num]), read_image(women_dir + file)), axis=0) for file in listdir(women_dir)])
		array = np.concatenate((array, new_men, new_women), axis=0)
	return array

dataset = read_images('../clean_reduced_dataset/')

In [107]:
arr = np.array([[[0,0,0], [200,20, 100]]])
cv2.inRange(arr,(0,0,80),(255,55,179))

array([[  0, 255]], dtype=uint8)

In [108]:
train, validation = train_test_split(dataset, test_size=0.1)
X = train[:, 1:]
y = train[:, 0]
X_val = validation[:, 1:]
y_val = validation[:, 0]

In [109]:
from IPython.display import display_html
from itertools import chain,cycle
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        # html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        # html_str+=f'<h2 style="text-align: center;">{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        # html_str+='</td></th>'
    display_html(html_str,raw=True)

John's Code

In [110]:
# count_arr = np.zeros((6,6))
def model_prediction(model, x_training, y_training, x_validation, y_validation):
	model.fit(x_training, y_training)
	
	prediction_training = np.array(np.round(model.predict(x_training)), dtype=np.int8)
	prediction_validation = np.array(np.round(model.predict(x_validation)), dtype=np.int8)
	
	acc_training = accuracy_score(y_training, prediction_training)
	acc_validation = accuracy_score(y_validation, prediction_validation)

	confusion_matrix_training = confusion_matrix(y_training, prediction_training)
	confusion_matrix_validation = confusion_matrix(y_validation, prediction_validation)
	# print("Confusion Matrix Training: \n", confusion_matrix_training)
	display_side_by_side(pd.DataFrame(confusion_matrix_training), pd.DataFrame(confusion_matrix_validation))
	# print("Confusion Matrix Validation: \n", confusion_matrix_validation)
	print("Accuracuy Score Training: ", acc_training)
	print("Accuracy Score Validation: ",acc_validation)

PCA

In [111]:
pca = PCA(n_components=30).fit(X)

In [112]:
# print('Random forest classifier')
# for i in range(1, 6):
# 	print(f'Run no.{i}:')
# 	model_prediction(RandomForestClassifier(n_estimators = 500, min_samples_split=10, max_features=None), pca.transform(X), y, pca.transform(X_val), y_val)
print('Bayes classifier')
for i in range(1, 6):
	print(f'Run no.{i}:')
	model_prediction(GaussianNB(), pca.transform(X), y, pca.transform(X_val), y_val)

# print('Random forest regressor')
# model_prediction(RandomForestRegressor(n_estimators = 500, min_samples_split=10, max_features=None), pca.transform(X), y, pca.transform(X_val), y_val)


# print('KNN')
# model_prediction(KNeighborsClassifier(n_neighbors=6), pca.transform(X), y, pca.transform(X_val), y_val)

Random forest classifier
Run no.1:


,0,1,2,3,4,5
0,254,0,0,0,0,0
1,0,273,0,0,0,0
2,0,1,278,0,0,0
3,0,0,0,254,0,0
4,1,0,0,0,275,0
5,1,0,0,1,0,276
,0,1,2,3,4,5
0,28,0,1,0,1,3
1,3,19,3,0,0,2
2,1,2,17,5,2,2


Accuracuy Score Training:  0.9975216852540273
Accuracy Score Validation:  0.6611111111111111
Run no.2:


,0,1,2,3,4,5
0,254,0,0,0,0,0
1,0,273,0,0,0,0
2,0,1,277,0,1,0
3,0,0,0,254,0,0
4,1,0,0,0,275,0
5,1,0,0,1,0,276
,0,1,2,3,4,5
0,28,0,1,0,1,3
1,3,19,3,0,0,2
2,1,2,18,3,2,3


Accuracuy Score Training:  0.9969021065675341
Accuracy Score Validation:  0.6611111111111111
Run no.3:


...

Accuracuy Score Training:  0.9981412639405205
Accuracy Score Validation:  0.6444444444444445
Run no.4:


,0,1,2,3,4,5
0,254,0,0,0,0,0
1,0,273,0,0,0,0
2,0,1,278,0,0,0
3,0,0,0,254,0,0
4,1,0,0,0,275,0
5,1,0,0,1,0,276
,0,1,2,3,4,5
0,29,0,0,1,0,3
1,3,17,4,1,0,2
2,1,2,16,6,2,2


Accuracuy Score Training:  0.9975216852540273
Accuracy Score Validation:  0.6444444444444445
Run no.5:


,0,1,2,3,4,5
0,254,0,0,0,0,0
1,0,273,0,0,0,0
2,0,1,277,0,1,0
3,0,0,0,254,0,0
4,1,0,0,0,275,0
5,1,0,0,1,0,276
,0,1,2,3,4,5
0,28,0,1,0,1,3
1,3,18,2,1,0,3
2,1,2,16,6,2,2


Accuracuy Score Training:  0.9969021065675341
Accuracy Score Validation:  0.65


In [ ]:
for i in range(10, 50, 2):
	print('*'* 20)
	print(f'PCA {i} components')
	pca = PCA(n_components=i).fit(X)
	print('Random forest')
	model_prediction(RandomForestClassifier(n_estimators = 500, min_samples_split=10, max_features=None), pca.transform(X), y, pca.transform(X_val), y_val)
	# print('KNN, N=5')
	# model_prediction(KNeighborsClassifier(n_neighbors=5), pca.transform(X), y, pca.transform(X_val), y_val)
	print('*'* 20)
	print('\n\n')